In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import string

In [ ]:
def wordopt(text):
  text = text.lower()
  text = re.sub('\[. *?\]','',text)
  text = re.sub("\\W"," ",text)
  text = re.sub('https?://\S+|www\.\S+', '', text)
  text = re.sub('<.*?›+', '', text)
  text = re.sub('[%s]' % re.escape(string.punctuation),'', text)
  text = re.sub( "\n",' ',text)
  text = re.sub('\w*\d\w*','',text)
  return text

In [ ]:
def get_data(search):
    url='https://en.wikipedia.org/wiki/'+str(search).strip()
    r=requests.get(url)
    soup=BeautifulSoup(r.content,'html.parser')
    complete=''
    for d in soup.find_all('p'):
      fine_tuned = wordopt(d.text)
      if(fine_tuned!=' '):
        complete = complete + fine_tuned
    return complete

In [ ]:
df_marvel = pd.read_csv('/content/marvel-wikia-data.csv')

In [ ]:
df_marvel = df_marvel[14000:]

In [ ]:
paras = []
final_df = pd.DataFrame([],columns=['elaborate_info'])
names = df_marvel['name']
# names = names[0:100]
paras=[]
for name in names:
  paras.append(get_data(re.sub("\(.*?\)","",name)))
temp = pd.DataFrame(paras, columns =['elaborate_info'])
final_df = pd.concat([final_df,temp],axis=0)
len(final_df)

2376

In [ ]:
df_marvel['info'] = list(final_df['elaborate_info'])
df_marvel = df_marvel.reset_index()
df_marvel.drop(['index'],axis=1,inplace=True)
temp_lst = list(df_marvel['info'])
indexs = []
invalid = ['other reasons this message may be displayed','there is currently no text in this page you can search for this page title in other pages or search the related logs but you do not have permission to create this page']
def check_valid(t):
  for i in invalid:
    if i == str(t).strip():
      return 0
  if 'refer to' in str(t) and len(str(t))<40:
    return 0
  return 1
for i in range(len(temp_lst)):
  if check_valid(temp_lst[i])==0:
    df_marvel.drop(i,axis=0,inplace=True)

In [ ]:
df_marvel.drop(['page_id','urlslug', 'ID','GSM', 'ALIVE','FIRST APPEARANCE', 'Year', 'APPEARANCES','SEX'],axis=1,inplace=True)

In [ ]:
df_marvel.to_csv('/content/operating_file6.csv',index=False)

In [ ]:
dfs = []
for i in range(1,7):
  df = pd.read_csv('/content/operating_file'+str(i)+'.csv')
  dfs.append(df)

In [ ]:
master_work = pd.concat(dfs)

In [ ]:
master_work = master_work.reset_index()
master_work.drop(['index'],axis=1,inplace=True)

In [ ]:
master_work

,name,ALIGN,EYE,HAIR,info
0,Spider-Man (Peter Parker),Good Characters,Hazel Eyes,Brown Hair,spider man is a superhero appearing in america...
1,Captain America (Steven Rogers),Good Characters,Blue Eyes,White Hair,captain america is a superhero appearing in am...
2,"Wolverine (James \""Logan\"" Howlett)",Neutral Characters,Blue Eyes,Black Hair,american wolverine g g luscus eurasian wo...
3,"Iron Man (Anthony \""Tony\"" Stark)",Good Characters,Blue Eyes,Black Hair,iron man is a superhero appearing in american ...
4,Thor (Thor Odinson),Good Characters,Blue Eyes,Blond Hair,thor from old norse þórr is a prominent god...
...,...,...,...,...,...
6471,Finch (Kate Bishop's neighbor) (Earth-616),Good Characters,Black Eyes,Bald,fringillinaecarduelinaeeuphoniinae the true fi...
6472,Jack O'Lantern (Impostor) (Earth-616),Bad Characters,Hazel Eyes,Bald,a jack o lantern or jack o lantern is a car...
6473,Marcus (Kate Bishop's neighbor) (Earth-616),Good Characters,Hazel Eyes,Bald,marcus markus márkus or mărcuș may refer to
6474,Thane (Thanos' son) (Earth-616),Good Characters,Blue Eyes,Bald,thane marathi ˈʈʰaːɳeː also known as than...


In [ ]:
master_work.to_csv('/content/nowNLP.csv',index=False)